In [ ]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Entradas de login e filial
email = input("Digite seu e-mail: ")
senha = input("Digite sua senha: ")
filial = input('Qual filial: ')

# Inicializa a lista para armazenar dados dos pedidos
order_data = []
detalhes_produtos = []

def login(driver):
    print("Realizando login...")
    driver.get("https://one.bees.com/order-management/closed-orders")
    WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="signInName"]'))).send_keys(email)
    driver.find_element(By.XPATH, '//*[@id="next"]').click()
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="password"]'))).send_keys(senha)
    driver.find_element(By.XPATH, '//*[@id="next"]').click()
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="single-spa-application:@admin-portal/main-mfe"]/section/header/div[1]/div/div[1]/div[3]/div[2]/button'))).click()
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[@id="radix-3-trigger-FAILED_DELIVERY"]'))).click()
 

def coletar_dados_pedidos(driver):
    """Coleta dados dos pedidos da página atual."""
    print("Coletando dados dos pedidos na página atual...")
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    pedidos = soup.findAll('tr', class_="c-fKvqNg c-fKvqNg-gsmDXe-rowClickable-true")

    for pedido in pedidos:
        colunas = pedido.findAll('td')
        texto_pedido = [coluna.get_text(strip=True) for coluna in colunas]
        numero_pedido = texto_pedido[0]  # Supondo que o número do pedido está na primeira coluna
        order_data.append(numero_pedido)

def navegar_paginas(driver):
    """Navega por todas as páginas de pedidos."""
    print("Navegando pelas páginas de pedidos...")
    for numero_pagina in range(1, 100):  # Ajuste o limite conforme necessário
        try:
            # Formata o XPath para a página atual
            page_xpath = f'//li[@title="page {numero_pagina}"]'
            coletar_dados_pedidos(driver)
            # Espera até que o botão "próxima página" esteja clicável
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, page_xpath))
            )
            next_button.click()
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//tbody[@role="rowgroup"]'))) 
        except Exception as e:
            print("Sem mais páginas")
            break

def carregar_pedidos_existentes(caminho):
    """Carrega os números dos pedidos existentes da planilha. Se a planilha não existir, cria um arquivo vazio."""
    try:
        # Tenta carregar o arquivo CSV existente
        df_existente = pd.read_csv(caminho)
        pedidos_existentes = set(df_existente["Número do pedido"].astype(str).str.strip().values)
        print("Pedidos existentes carregados da planilha.")
    except FileNotFoundError:
        # Se o arquivo não for encontrado, cria um DataFrame vazio
        pedidos_existentes = set()  # Conjunto vazio para pedidos
        print("Planilha não encontrada. Criando planilha vazia.")
        
        # Cria a estrutura inicial da planilha com os cabeçalhos
        df_existente = pd.DataFrame(columns=["CD",
            "Número do pedido", "Nome do produto", "SKU", "Quantidade não entregue", 
            "Preço", "Forma de Pagamento", "Endereço de Entrega", "Documento", 
            "Motivo", "Total do pedido", "Data"
        ])
        # Cria o arquivo vazio (caso não exista)
        df_existente.to_csv(caminho, index=False, encoding='utf-8-sig')
    
    return pedidos_existentes, df_existente


def coletar_detalhes_pedido(driver, order):
    """Coleta detalhes de um pedido específico."""
    print(f"Coletando detalhes do pedido: {order}")
    time.sleep(10)
    driver.get(f"https://one.bees.com/order-management/closed-orders/{order}")
    time.sleep(10)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//tbody[@role="rowgroup"]')))
    tbody = driver.find_element(By.XPATH, '//tbody[@role="rowgroup"]')
    produtos = tbody.find_elements(By.XPATH, './/tr[@role="row"]')
    detalhes_pedido = driver.find_element(By.CLASS_NAME, 'c-jhfORj')

    total_elemento = driver.find_element(By.XPATH, '//div[@data-testid="order-total-value"]')
    total = total_elemento.find_element(By.TAG_NAME, 'div').text.strip()

    for produto in produtos:

        cd = detalhes_pedido.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[4]/div[2]/p[2]').text.strip()
        nome_sku = produto.find_element(By.XPATH, './/td[1]//p[1]').text.strip()
        sku = produto.find_element(By.XPATH, './/td[1]//p[2]').text.strip()
        quantidade_nao_entregue = produto.find_element(By.XPATH, './/td[2]').text.strip()
        preco = produto.find_element(By.XPATH, './/td[3]').text.strip()
        forma_pgto = detalhes_pedido.find_element(By.XPATH, '//p[@data-testid="method-0"]').text.strip()
        endereco_entrega = detalhes_pedido.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[6]/div[3]/div[1]/div[5]/p[3]').text.strip()
        documento = detalhes_pedido.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[6]/div[3]/div[1]/div[2]/p[2]').text.strip()
        motivo = detalhes_pedido.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[6]/div[3]/div[2]/div[4]/p[2]').text.strip()
        data = detalhes_pedido.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[6]/div[3]/div[2]/div[1]/p[2]').text.strip()
        detalhes_produtos.append({
            "CD": cd,
            "Número do pedido": order,
            "Nome do produto": nome_sku,
            "SKU": sku,
            "Quantidade não entregue": quantidade_nao_entregue,
            "Preço": preco,
            "Forma de Pagamento": forma_pgto,
            "Endereço de Entrega": endereco_entrega,
            "Documento": documento,
            "Motivo": motivo,
            "Total do pedido": total,
            "Data": data
        })

def pedidos(driver, pedidos_existentes, df_existente):
    print("Iniciando a coleta de detalhes dos pedidos...")
    pedidos_novos = []  # Para armazenar pedidos que não estão na planilha
    for order in order_data:
        order = order.strip()  # Remove espaços em branco ao redor
        print(f"Verificando o pedido: {order}")  # Debug: Verifica cada pedido
        if order in pedidos_existentes:
            print(f"Pedido {order} já existe na planilha. Pulando...")
            continue  # Pula para o próximo pedido se já estiver na planilha
        else:
            pedidos_novos.append(order)  # Adiciona pedidos novos à lista

    print("Pedidos que não estão na planilha:", pedidos_novos)  # Imprime os pedidos novos

    # Ordena os pedidos em ordem decrescente
    pedidos_novos.sort(reverse=True)  # Ordena os pedidos do maior para o menor

    # Para cada pedido, coleta os detalhes e salva diretamente no CSV
    for order in pedidos_novos:
        coletar_detalhes_pedido(driver, order)  # Coleta detalhes apenas se o pedido não está na planilha

        # Cria o DataFrame com os detalhes coletados
        df_novo_pedido = pd.DataFrame(detalhes_produtos[-1:], columns=df_existente.columns)  # Adiciona um único pedido

        # Adiciona os dados diretamente ao arquivo CSV (sem sobrescrever)
        caminho = f'Pedidos_Nao_Entregues_{filial}.csv'
        df_novo_pedido.to_csv(caminho, mode='a', index=False, header=False, encoding='utf-8-sig')

        # Limpa a lista de detalhes para o próximo pedido
        detalhes_produtos.clear()

    print("Coleta e salvamento dos pedidos finalizados!")

    df_novos = pd.DataFrame(detalhes_produtos)
    df_final = pd.concat([df_existente, df_novos], ignore_index=True)  # Adiciona novos pedidos
    caminho = f'Pedidos_Nao_Entregues_{filial}.csv'
    df_final.to_csv(caminho, index=False, encoding='utf-8-sig')
    print("Detalhes salvos com sucesso!")


# Inicializa o driver
driver = webdriver.Chrome()
driver.maximize_window()
# Carrega os pedidos existentes
caminho = f'Pedidos_Nao_Entregues_{filial}.csv'
pedidos_existentes, df_existente = carregar_pedidos_existentes(caminho)

# Realiza o login
login(driver)

# Navega pelas páginas de pedidos
navegar_paginas(driver)

# Coleta os dados dos pedidos
pedidos(driver, pedidos_existentes, df_existente)

# Fecha o navegador
driver.quit()


Pedidos existentes carregados da planilha.
Realizando login...
Navegando pelas páginas de pedidos...
Coletando dados dos pedidos na página atual...
Coletando dados dos pedidos na página atual...
Coletando dados dos pedidos na página atual...
Coletando dados dos pedidos na página atual...
Coletando dados dos pedidos na página atual...
Coletando dados dos pedidos na página atual...
Coletando dados dos pedidos na página atual...
Coletando dados dos pedidos na página atual...
Coletando dados dos pedidos na página atual...
Coletando dados dos pedidos na página atual...
Sem mais páginas
Iniciando a coleta de detalhes dos pedidos...
Verificando o pedido: 9196160787
Pedido 9196160787 já existe na planilha. Pulando...
Verificando o pedido: 9194512877
Verificando o pedido: 9196101458
Pedido 9196101458 já existe na planilha. Pulando...
Verificando o pedido: 9194531804
Verificando o pedido: 9196145671
Pedido 9196145671 já existe na planilha. Pulando...
Verificando o pedido: 9195542138
Verificando 